Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, levene, f_oneway, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp

# **Multi-task**

Tải dữ liệu (Chỉ lấy 3 cột: Dice, F1-Score, Inference_Time)

In [2]:
MTL_ConV_Capsnet = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\MTL\MTL_ConV-Capsnet.xlsx", sheet_name='Sheet1', header=0)
MTL_ConV_FC = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\MTL\MTL_ConV-FC.xlsx", sheet_name='Sheet1', header=0)
MTL_Deform_Capsnet = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\MTL\MTL_Deform-Capsnet.xlsx", sheet_name='Sheet1', header=0)
MTL_Deform_FC = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\MTL\MTL_Deform-FC.xlsx", sheet_name='Sheet1', header=0)

MTL_ConV_Capsnet = MTL_ConV_Capsnet.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]
MTL_ConV_FC = MTL_ConV_FC.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]
MTL_Deform_Capsnet = MTL_Deform_Capsnet.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]
MTL_Deform_FC = MTL_Deform_FC.loc[:,['Dice','F1-Score','Inference_Time', 'Accuracy']]

## **1. Kiểm định cho SEGMENTATION**

Bảng tổng hợp các chỉ số Dice từ 4 mô hình MTL

In [3]:
MTL_Dice = pd.DataFrame(
    {
    'ConV_Capsnet': MTL_ConV_Capsnet['Dice'],
    'ConV_FC': MTL_ConV_FC['Dice'],
    'Deform_Capsnet': MTL_Deform_Capsnet['Dice'],
    'Deform_FC': MTL_Deform_FC['Dice']
    })

MTL_Dice.head()

,ConV_Capsnet,ConV_FC,Deform_Capsnet,Deform_FC
0,0.755100,0.762538,0.713969,0.733115
1,0.765757,0.735527,0.712976,0.738174
2,0.755895,0.742612,0.712917,0.687125
3,0.750135,0.719212,0.631400,0.645657
4,0.770810,0.769290,0.655200,0.726202


### **1.1. Kiểm định Phân phối chuẩn (Shapiro-Wilk)**

In [4]:

alpha = 0.05
is_normal_seg = True

for col in MTL_Dice.columns:
    stat_seg, p_shapiro_seg = shapiro(MTL_Dice[col])
    print(f'{col}: Statistic={stat_seg:.4f}, p-value={p_shapiro_seg:.4f}')
    
    if p_shapiro_seg > alpha:
        print(f'{col} CÓ THỂ COI là phân phối chuẩn (fail to reject H0)\n')
    else:
        print(f'{col} KHÔNG PHẢI là phân phối chuẩn (reject H0)\n')
        is_normal_seg = False


ConV_Capsnet: Statistic=0.8520, p-value=0.0007
ConV_Capsnet KHÔNG PHẢI là phân phối chuẩn (reject H0)

ConV_FC: Statistic=0.9510, p-value=0.1803
ConV_FC CÓ THỂ COI là phân phối chuẩn (fail to reject H0)

Deform_Capsnet: Statistic=0.8884, p-value=0.0044
Deform_Capsnet KHÔNG PHẢI là phân phối chuẩn (reject H0)

Deform_FC: Statistic=0.9432, p-value=0.1111
Deform_FC CÓ THỂ COI là phân phối chuẩn (fail to reject H0)



### **1.2. Kiểm định Kruskal-Wallis H**

In [5]:
# Nếu dữ liệu phân phối không chuẩn, thực hiện kiểm định Kruskal-Wallis
if is_normal_seg == False:
    seg_announce = ' MULTI-TASK LEARNING (MTL) - ĐÁNH GIÁ DICE SCORE CHO SEGMENTATION: '
    Seg_Announce = seg_announce.center(80, '-')
    print(Seg_Announce)
    print('\nVì dữ liệu vi phạm giả định phân phối chuẩn, nên chuyển sang Kiểm định phi tham số Kruskal-Wallis.\n')
    
    # Kiểm định Kruskal-Wallis
    h_stat, p_kruskal = kruskal(
    MTL_Dice['ConV_Capsnet'],
    MTL_Dice['ConV_FC'],
    MTL_Dice['Deform_Capsnet'],
    MTL_Dice['Deform_FC']
    )

    # In kết quả kiểm định Kruskal-Wallis
    print("-- Kiểm định Kruskal-Wallis cho MTL-Dice: ---")
    print(f"Thống kê H (H-statistic): {h_stat:.4f}")
    print(f"Giá trị p (p-value): {p_kruskal:.4f}")

    if p_kruskal < alpha:
        print("\nCó bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).")
        print("Tiến hành kiểm định Dunn's Post-Hoc để so sánh từng cặp mô hình.")

        # Chuyển dữ liệu sang dạng "long" để dùng cho posthoc_dunn
        dice_long = pd.melt(MTL_Dice.reset_index(), id_vars=['index'], value_vars=MTL_Dice.columns,
                            var_name='Model', value_name='Dice')
        dunn_result = sp.posthoc_dunn(dice_long, group_col='Model', val_col='Dice', p_adjust='bonferroni')

        # In kết quả kiểm định Dunn's Post-Hoc
        print("\n--- Bảng kết quả kiểm định Dunn's Post-Hoc (đã hiệu chỉnh Bonferroni) ---")
        print(dunn_result.round(4))

        print("\n--- Kết quả trung bình Dice Score cho từng mô hình ---")
        print(f'ConV_Capsnet:\t {MTL_ConV_Capsnet["Dice"].mean():.4f}')
        print(f'ConV_FC:\t {MTL_ConV_FC["Dice"].mean():.4f}')
        print(f'Deform_Capsnet:\t {MTL_Deform_Capsnet["Dice"].mean():.4f}') 
        print(f'Deform_FC:\t {MTL_Deform_FC["Dice"].mean():.4f}')
    
    else:
        print("\nKhông có đủ bằng chứng để kết luận có sự khác biệt về hiệu suất giữa các mô hình (p >= 0.05).")
        print("Dừng phân tích tại đây.")

significant_pairs = []
non_significant_pairs = []

for i in range(len(dunn_result.index)):
    for j in range(i+1, len(dunn_result.columns)):
        p = dunn_result.iloc[i, j]
        pair = (dunn_result.index[i], dunn_result.columns[j])
        mean1 = MTL_Dice[pair[0]].mean()  # Nếu dùng Dice thì thay MTL_F1 bằng MTL_Dice
        mean2 = MTL_Dice[pair[1]].mean()
        entry = {
            'Model 1': pair[0],
            'Model 2': pair[1],
            'Mean Model 1': round(mean1, 4),
            'Mean Model 2': round(mean2, 4),
            'p-value': round(p, 4),
            'Ý nghĩa': 'Có' if p < 0.05 else 'Không'
        }
        if p < 0.05:
            significant_pairs.append(entry)
        else:
            non_significant_pairs.append(entry)

seg_pairs = pd.DataFrame(significant_pairs + non_significant_pairs)
        

------ MULTI-TASK LEARNING (MTL) - ĐÁNH GIÁ DICE SCORE CHO SEGMENTATION: -------

Vì dữ liệu vi phạm giả định phân phối chuẩn, nên chuyển sang Kiểm định phi tham số Kruskal-Wallis.

-- Kiểm định Kruskal-Wallis cho MTL-Dice: ---
Thống kê H (H-statistic): 91.1160
Giá trị p (p-value): 0.0000

Có bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).
Tiến hành kiểm định Dunn's Post-Hoc để so sánh từng cặp mô hình.

--- Bảng kết quả kiểm định Dunn's Post-Hoc (đã hiệu chỉnh Bonferroni) ---
                ConV_Capsnet  ConV_FC  Deform_Capsnet  Deform_FC
ConV_Capsnet             1.0      1.0          0.0000     0.0000
ConV_FC                  1.0      1.0          0.0000     0.0000
Deform_Capsnet           0.0      0.0          1.0000     0.1762
Deform_FC                0.0      0.0          0.1762     1.0000

--- Kết quả trung bình Dice Score cho từng mô hình ---
ConV_Capsnet:	 0.7648
ConV_FC:	 0.7509
Deform_Capsnet:	 0.6496
Deform_FC:	 0.6881


## **2. Kiểm định cho CLASSIFICATION**

Bảng tổng hợp các chỉ số F1-Score từ bốn mô hình MTL

In [6]:
MTL_F1 = pd.DataFrame(
    {
    'ConV_Capsnet': MTL_ConV_Capsnet['F1-Score'],
    'ConV_FC': MTL_ConV_FC['F1-Score'],
    'Deform_Capsnet': MTL_Deform_Capsnet['F1-Score'],
    'Deform_FC': MTL_Deform_FC['F1-Score']
    })

MTL_F1.head()

,ConV_Capsnet,ConV_FC,Deform_Capsnet,Deform_FC
0,0.870851,0.892702,0.830406,0.879420
1,0.832843,0.884363,0.894044,0.889034
2,0.830382,0.897668,0.848814,0.901713
3,0.830382,0.908741,0.781300,0.881259
4,0.859590,0.865144,0.791400,0.858593


### **2.1. Kiểm định Phân phối chuẩn (Shapiro-Wilk)**

In [7]:
is_normal_cls = True

for col in MTL_F1.columns:
    stat_cls, p_shapiro_cls = shapiro(MTL_F1[col])
    print(f'{col}: Statistic={stat_cls:.4f}, p-value={p_shapiro_cls:.4f}')
    
    if p_shapiro_cls > alpha:
        print(f'{col} CÓ THỂ COI là phân phối chuẩn (fail to reject H0)\n')
    else:
        print(f'{col} KHÔNG PHẢI là phân phối chuẩn (reject H0)\n')
        is_normal_cls = False

ConV_Capsnet: Statistic=0.9275, p-value=0.0422
ConV_Capsnet KHÔNG PHẢI là phân phối chuẩn (reject H0)

ConV_FC: Statistic=0.9674, p-value=0.4697
ConV_FC CÓ THỂ COI là phân phối chuẩn (fail to reject H0)

Deform_Capsnet: Statistic=0.9216, p-value=0.0295
Deform_Capsnet KHÔNG PHẢI là phân phối chuẩn (reject H0)

Deform_FC: Statistic=0.9538, p-value=0.2139
Deform_FC CÓ THỂ COI là phân phối chuẩn (fail to reject H0)



### **2.2. Kiểm định Kruskal-Wallis**

In [8]:
# Nếu dữ liệu phân phối không chuẩn, thực hiện kiểm định Kruskal-Wallis
if is_normal_cls == False:
    cls_announce = ' MULTI-TASK LEARNING (MTL) - ĐÁNH GIÁ F1 SCORE CHO CLASSIFICATION: '
    Cls_Announce = cls_announce.center(80, '-')
    print(Cls_Announce)
    print('\nVì dữ liệu vi phạm giả định phân phối chuẩn, nên chuyển sang Kiểm định phi tham số Kruskal-Wallis.\n')
    
    # Kiểm định Kruskal-Wallis
    h_stat, p_kruskal = kruskal(
    MTL_F1['ConV_Capsnet'],
    MTL_F1['ConV_FC'],
    MTL_F1['Deform_Capsnet'],
    MTL_F1['Deform_FC']
    )

    # In kết quả kiểm định Kruskal-Wallis
    print("-- Kiểm định Kruskal-Wallis cho MTL-F1_Score: ---")
    print(f"Thống kê H (H-statistic): {h_stat:.4f}")
    print(f"Giá trị p (p-value): {p_kruskal:.4f}")

    if p_kruskal < alpha:
        print("\nCó bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).")
        print("Tiến hành kiểm định Dunn's Post-Hoc để so sánh từng cặp mô hình.")

        # Chuyển dữ liệu sang dạng "long" để dùng cho posthoc_dunn
        f1_long = pd.melt(MTL_F1.reset_index(), id_vars=['index'], value_vars=MTL_F1.columns,
                            var_name='Model', value_name='F1-Score')
        dunn_result = sp.posthoc_dunn(f1_long, group_col='Model', val_col='F1-Score', p_adjust='bonferroni')

        # In kết quả kiểm định Dunn's Post-Hoc
        print("\n--- Bảng kết quả kiểm định Dunn's Post-Hoc (đã hiệu chỉnh Bonferroni) ---")
        print(dunn_result.round(4))

        print("\n--- Kết quả trung bình F1-Score cho từng mô hình ---")
        print(f'ConV_Capsnet:\t {MTL_ConV_Capsnet["F1-Score"].mean():.4f}')
        print(f'ConV_FC:\t {MTL_ConV_FC["F1-Score"].mean():.4f}')
        print(f'Deform_Capsnet:\t {MTL_Deform_Capsnet["F1-Score"].mean():.4f}') 
        print(f'Deform_FC:\t {MTL_Deform_FC["F1-Score"].mean():.4f}')
    
    else:
        print("\nKhông có đủ bằng chứng để kết luận có sự khác biệt về hiệu suất giữa các mô hình (p >= 0.05).")
        print("Dừng phân tích tại đây.")

significant_pairs = []
non_significant_pairs = []

for i in range(len(dunn_result.index)):
    for j in range(i+1, len(dunn_result.columns)):
        p = dunn_result.iloc[i, j]
        pair = (dunn_result.index[i], dunn_result.columns[j])
        mean1 = MTL_F1[pair[0]].mean()  # Nếu dùng Dice thì thay MTL_F1 bằng MTL_Dice
        mean2 = MTL_F1[pair[1]].mean()
        entry = {
            'Model 1': pair[0],
            'Model 2': pair[1],
            'Mean Model 1': round(mean1, 4),
            'Mean Model 2': round(mean2, 4),
            'p-value': round(p, 4),
            'Ý nghĩa': 'Có' if p < 0.05 else 'Không',
        }
        if p < 0.05:
            significant_pairs.append(entry)
        else:
            non_significant_pairs.append(entry)

cls_pairs = pd.DataFrame(significant_pairs + non_significant_pairs)
        

------ MULTI-TASK LEARNING (MTL) - ĐÁNH GIÁ F1 SCORE CHO CLASSIFICATION: -------

Vì dữ liệu vi phạm giả định phân phối chuẩn, nên chuyển sang Kiểm định phi tham số Kruskal-Wallis.

-- Kiểm định Kruskal-Wallis cho MTL-F1_Score: ---
Thống kê H (H-statistic): 44.2303
Giá trị p (p-value): 0.0000

Có bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).
Tiến hành kiểm định Dunn's Post-Hoc để so sánh từng cặp mô hình.

--- Bảng kết quả kiểm định Dunn's Post-Hoc (đã hiệu chỉnh Bonferroni) ---
                ConV_Capsnet  ConV_FC  Deform_Capsnet  Deform_FC
ConV_Capsnet          1.0000     0.28          0.0002        1.0
ConV_FC               0.2800     1.00          0.0000        1.0
Deform_Capsnet        0.0002     0.00          1.0000        0.0
Deform_FC             1.0000     1.00          0.0000        1.0

--- Kết quả trung bình F1-Score cho từng mô hình ---
ConV_Capsnet:	 0.8612
ConV_FC:	 0.8738
Deform_Capsnet:	 0.8117
Deform_FC:	 0.8650


In [9]:
seg_pairs

,Model 1,Model 2,Mean Model 1,Mean Model 2,p-value,Ý nghĩa
0,ConV_Capsnet,Deform_Capsnet,0.7648,0.6496,0.0000,Có
1,ConV_Capsnet,Deform_FC,0.7648,0.6881,0.0000,Có
2,ConV_FC,Deform_Capsnet,0.7509,0.6496,0.0000,Có
3,ConV_FC,Deform_FC,0.7509,0.6881,0.0000,Có
4,ConV_Capsnet,ConV_FC,0.7648,0.7509,1.0000,Không
5,Deform_Capsnet,Deform_FC,0.6496,0.6881,0.1762,Không


In [10]:
cls_pairs


,Model 1,Model 2,Mean Model 1,Mean Model 2,p-value,Ý nghĩa
0,ConV_Capsnet,Deform_Capsnet,0.8612,0.8117,0.0002,Có
1,ConV_FC,Deform_Capsnet,0.8738,0.8117,0.0000,Có
2,Deform_Capsnet,Deform_FC,0.8117,0.8650,0.0000,Có
3,ConV_Capsnet,ConV_FC,0.8612,0.8738,0.2800,Không
4,ConV_Capsnet,Deform_FC,0.8612,0.8650,1.0000,Không
5,ConV_FC,Deform_FC,0.8738,0.8650,1.0000,Không


### **Accuracy**

In [11]:
MTL_Acc = pd.DataFrame(
    {
    'ConV_Capsnet': MTL_ConV_Capsnet['Accuracy'],
    'ConV_FC': MTL_ConV_FC['Accuracy'],
    'Deform_Capsnet': MTL_Deform_Capsnet['Accuracy'],
    'Deform_FC': MTL_Deform_FC['Accuracy']
    })

MTL_Acc.head()

,ConV_Capsnet,ConV_FC,Deform_Capsnet,Deform_FC
0,0.884662,0.875023,0.824424,0.859150
1,0.824120,0.884115,0.897342,0.872377
2,0.836739,0.887946,0.840327,0.883567
3,0.836739,0.831205,0.779400,0.867390
4,0.864684,0.851487,0.813800,0.873502


In [12]:
is_normal_acc = True

for col in MTL_Acc.columns:
    stat_acc, p_shapiro_acc = shapiro(MTL_Acc[col])
    print(f'{col}: statistic={stat_acc:.4f}, p-value={p_shapiro_acc:.4f}')
    
    if p_shapiro_acc > alpha:
        print(f'{col} CÓ THỂ COI là phân phối chuẩn (fail to reject H0)\n')
    else:
        print(f'{col} KHÔNG PHẢI là phân phối chuẩn (reject H0)\n')
        is_normal_acc = False

ConV_Capsnet: statistic=0.9377, p-value=0.0788
ConV_Capsnet CÓ THỂ COI là phân phối chuẩn (fail to reject H0)

ConV_FC: statistic=0.9673, p-value=0.4694
ConV_FC CÓ THỂ COI là phân phối chuẩn (fail to reject H0)

Deform_Capsnet: statistic=0.9433, p-value=0.1113
Deform_Capsnet CÓ THỂ COI là phân phối chuẩn (fail to reject H0)

Deform_FC: statistic=0.9871, p-value=0.9672
Deform_FC CÓ THỂ COI là phân phối chuẩn (fail to reject H0)



In [13]:
# Nếu dữ liệu phân phối chuẩn, thực hiện kiểm định ANOVA
if is_normal_acc == True:
    cls_announce = ' MULTI-TASK LEARNING (MTL) - ĐÁNH GIÁ ACCURACY CHO CLASSIFICATION: '
    Cls_Announce = cls_announce.center(80, '-')
    print(Cls_Announce)
    print('\nVì dữ liệu tuân theo giả định phân phối chuẩn, nên chuyển sang Kiểm định Đồng nhất phương sai Levene.\n')
    
    # Kiểm định Đồng nhất phương sai Levene
    stat_levene, p_levene = levene(
        MTL_Acc['ConV_Capsnet'],
        MTL_Acc['ConV_FC'],
        MTL_Acc['Deform_Capsnet'],
        MTL_Acc['Deform_FC']
    )

    print("--- Kiểm định Đồng nhất phương sai Levene cho MTL-Accuracy: ---")
    print(f"Thống kê Levene: {stat_levene:.4f}")
    print(f"Giá trị p (p-value): {p_levene:.4f}")

    if p_levene > alpha:
        print('\nKết quả Levene cho thấy phương sai giữa các nhóm là đồng nhất (p-value > 0.05).')
        print('Tiến hành kiểm định ANOVA một chiều để so sánh từng cặp mô hình.\n')

        # Kiểm định ANOVA một chiều
        f_stat, p_anova = f_oneway(
            MTL_Acc['ConV_Capsnet'],
            MTL_Acc['ConV_FC'],
            MTL_Acc['Deform_Capsnet'],
            MTL_Acc['Deform_FC']
        )
        print("--- Kết quả kiểm định ANOVA một chiều cho MTL-Accuracy: ---")
        print(f"Thống kê F: {f_stat:.4f}, p-value: {p_anova:.4f}")

        if p_anova < alpha:
            print("\nCó bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).")
            print("Tiến hành kiểm định Tukey's HSD Post-Hoc để so sánh từng cặp mô hình.")

            # Chuyển dữ liệu sang dạng "long" để dùng cho posthoc_tukey
            acc_long = pd.melt(MTL_Acc.reset_index(), id_vars=['index'], value_vars=MTL_Acc.columns)
            acc_long.columns = ['Index', 'Model', 'Accuracy']
            # Kiểm định Tukey's HSD Post-Hoc
            tukey_result = pairwise_tukeyhsd(endog=acc_long['Accuracy'], groups=acc_long['Model'], alpha=alpha)

            # In kết quả kiểm định Tukey's HSD Post-Hoc
            print("\n--- Bảng kết quả kiểm định Tukey's HSD Post-Hoc ---")
            print(tukey_result)

            print("\n--- Kết quả trung bình Accuracy cho từng mô hình ---")
            print(f'ConV_Capsnet:\t {MTL_ConV_Capsnet["Accuracy"].mean():.4f}')
            print(f'ConV_FC:\t {MTL_ConV_FC["Accuracy"].mean():.4f}')
            print(f'Deform_Capsnet:\t {MTL_Deform_Capsnet["Accuracy"].mean():.4f}') 
            print(f'Deform_FC:\t {MTL_Deform_FC["Accuracy"].mean():.4f}')

------ MULTI-TASK LEARNING (MTL) - ĐÁNH GIÁ ACCURACY CHO CLASSIFICATION: -------

Vì dữ liệu tuân theo giả định phân phối chuẩn, nên chuyển sang Kiểm định Đồng nhất phương sai Levene.

--- Kiểm định Đồng nhất phương sai Levene cho MTL-Accuracy: ---
Thống kê Levene: 1.8093
Giá trị p (p-value): 0.1493

Kết quả Levene cho thấy phương sai giữa các nhóm là đồng nhất (p-value > 0.05).
Tiến hành kiểm định ANOVA một chiều để so sánh từng cặp mô hình.

--- Kết quả kiểm định ANOVA một chiều cho MTL-Accuracy: ---
Thống kê F: 29.3151, p-value: 0.0000

Có bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).
Tiến hành kiểm định Tukey's HSD Post-Hoc để so sánh từng cặp mô hình.

--- Bảng kết quả kiểm định Tukey's HSD Post-Hoc ---
        Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1         group2     meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------
  ConV_Capsnet        ConV_FC   0.0089 0